In [1]:
import pyVHR as vhr
import numpy as np
import pandas as pd
import os

from pyVHR.analysis.pipeline import Pipeline

dataset_name = 'lgi_ppgi'          
video_DIR = '../datasets/LGI-PPGI/' 
BVP_DIR = '../datasets/LGI-PPGI/'    # dir containing BVPs GT

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR= video_DIR, BVPdataDIR= BVP_DIR)
allvideo = dataset.videoFilenames
allground = dataset.sigFilenames
pipeline = Pipeline()

titles = []
for i in allvideo:
    txt = i.split('/')[4]
    txt_ = txt.split('_')
    titles.append(( txt_[0],txt_[1]))
# get the name of i video: titles[i][0]
# get the activity of i video: tittles[i][1]

2022-10-14 00:30:05.113226: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Métricas

* Generar un CSV con la información de cada video del dataset LGI-PPGI.
* Columnas:
    - time (seconds).
    - ground truth (BPM).
    - predicted (BPM, uncert).
    - roi methods (extraction, approach) = convex hull, holistic.
    - rPPG method.
* Asegurar unificación de time.

In [19]:
def get_video_data(met, app, video_index):
    BPMs_predicted = []
    times_predicted = []
    uncerts_predicted = []

    bpmgt, timegt = vhr.datasets.lgi_ppgi.LGI_PPGI(videodataDIR= allvideo[video_index], BVPdataDIR= allground[video_index]).readSigfile(filename= allground[video_index]).getBPM()
    timegt = [int(item) for item in timegt]

    for i in allvideo:
        txt = i.split('/')[4]
        txt_ = txt.split('_')
        title = ( txt_[0],txt_[1]) 

    methods = ['cupy_POS', 'torch_CHROM', 'cupy_CHROM']

    for method in methods:
        timepd, bpmpd, uncertpd = pipeline.run_on_video(allvideo[video_index],cuda = True, roi_method=met, roi_approach=app, method= method,  verb=False )
        BPMs_predicted.append(bpmpd)
        times_predicted.append(timepd)
        uncerts_predicted.append(uncertpd)

    data_cupy_POS = {
        'time': times_predicted[0], 
        'BPM predicted': BPMs_predicted[0],
        'uncert': uncerts_predicted[0],
        'rPPG method' : methods[0],
        }
    data_torch_CHROM = {
        'time': times_predicted[1], 
        'BPM predicted': BPMs_predicted[1],
        'uncert': uncerts_predicted[1],
        'rPPG method' : methods[1]
        }
    data_cupy_CHROM = {
        'time': times_predicted[2], 
        'BPM predicted': BPMs_predicted[2],
        'uncert': uncerts_predicted[2],
        'rPPG method' : methods[2]
        }
    data_gt = {'extraction': met, 'approach': app, 'BPM gt': bpmgt, 'time gt': timegt}
    
    return(data_cupy_CHROM, data_torch_CHROM, data_cupy_POS, data_gt)

def get_video_dataframes(data_cupy_CHROM, data_torch_CHROM, data_cupy_POS, data_gt):
    frame_cupy_POS = pd.DataFrame(data_cupy_POS)
    frame_torch_POS = pd.DataFrame(data_torch_CHROM)
    frame_cupy_CHROM = pd.DataFrame(data_cupy_CHROM)
    frame_extraction = pd.DataFrame(data_gt, index = data_gt['time gt'])
    return (frame_cupy_CHROM, frame_torch_POS, frame_cupy_POS, frame_extraction)

In [20]:
dir = './metrics_datasets/'

#video = 0

for video in range(0,len(allvideo)):
    try:
        person = titles[video][0]
        activity = titles[video][1]
        print("----Procesing video n°"+str(video)+" of "+str(range(21,len(allvideo))[1]))
        print("Generating data about "+person+' '+activity+'...') 

        #Generate data from prediction and GT
        data_cupy_CHROM, data_torch_CHROM, data_cupy_POS, data_gt = get_video_data('convexhull','holistic', video_index= video)

        #Create DataFrame from previous data
        frame_cupy_CHROM, frame_torch_CHROM, frame_cupy_POS, frame_gt = get_video_dataframes(data_cupy_CHROM, data_torch_CHROM, data_cupy_POS, data_gt)

        #Create CSV for each DataFrame
        if not os.path.exists(dir):
            os.mkdir(dir)

        rPPG_method = f'{frame_cupy_CHROM=}'.split('=')[0].split('_')
        frame_cupy_CHROM.to_csv(dir+person+'_'+activity+'('+ rPPG_method[1]+rPPG_method[2] +')'+'.csv', index= False)

        rPPG_method = f'{frame_torch_CHROM=}'.split('=')[0].split('_')
        frame_torch_CHROM.to_csv(dir+person+'_'+activity+'('+ rPPG_method[1]+rPPG_method[2] +')'+'.csv', index= False)

        rPPG_method = f'{frame_cupy_POS=}'.split('=')[0].split('_')
        frame_cupy_POS.to_csv(dir+person+'_'+activity+'('+ rPPG_method[1]+rPPG_method[2] +')'+'.csv', index= False)

        frame_gt.to_csv(dir+person+'_'+activity+'(extraction_GT)'+'.csv', index= False)
    except:
        print("Error generando datos para "+person+' '+activity)
        pass



----Procesing video n°0 of 22
Generating data about alex talk...
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
----Procesing video n°1 of 22
Generating data about alex resting...
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
----Procesing video n°2 of 22
Generating data about alex rotation...
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
----Procesing video n°3 of 22
Generating data about alex gym...
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1


# Testing Code

In [17]:
video = 1
person = titles[video][0]
activity = titles[video][1]
print("----Procesing video n°"+str(video)+" of "+str(range(21,len(allvideo))[1]))
print("Generating data about "+person+' '+activity+'...') 

#Generate data from prediction and GT
data_cupy_CHROM, data_torch_CHROM, data_cupy_POS, data_gt = get_video_data('convexhull','holistic', video_index= video)

#Create DataFrame from previous data
frame_cupy_CHROM, frame_torch_CHROM, frame_cupy_POS, frame_gt = get_video_dataframes(data_cupy_CHROM, data_torch_CHROM, data_cupy_POS, data_gt)


----Procesing video n°1 of 22
Generating data about alex resting...
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060
# CUDA devices:  1
# device number  0 :  NVIDIA GeForce RTX 3060


In [15]:
dir = './metrics_datasets/'
if not os.path.exists(dir):
    os.mkdir(dir)

rPPG_method = f'{frame_cupy_CHROM=}'.split('=')[0].split('_')
frame_cupy_CHROM.to_csv(dir+person+'_'+activity+'('+ rPPG_method[1]+rPPG_method[2] +')'+'.csv', index= False)

rPPG_method = f'{frame_torch_CHROM=}'.split('=')[0].split('_')
frame_torch_CHROM.to_csv(dir+person+'_'+activity+'('+ rPPG_method[1]+rPPG_method[2] +')'+'.csv', index= False)

rPPG_method = f'{frame_cupy_POS=}'.split('=')[0].split('_')
frame_cupy_POS.to_csv(dir+person+'_'+activity+'('+ rPPG_method[1]+rPPG_method[2] +')'+'.csv', index= False)

frame_gt.to_csv(dir+person+'_'+activity+'(extraction_GT)'+'.csv', index= False)

In [18]:
frame_cupy_CHROM

,time,BPM predicted,uncert,rPPG method
0,3.0,67.382812,0.0,cupy_CHROM
1,4.0,65.917969,0.0,cupy_CHROM
2,5.0,66.650391,0.0,cupy_CHROM
3,6.0,68.115234,0.0,cupy_CHROM
4,7.0,68.115234,0.0,cupy_CHROM
...,...,...,...,...
63,66.0,67.382812,0.0,cupy_CHROM
64,67.0,65.185547,0.0,cupy_CHROM
65,68.0,62.255859,0.0,cupy_CHROM
66,69.0,60.791016,0.0,cupy_CHROM
